## Import Libraries

In [1]:
import os
import torch
import torch.nn as nn
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader, random_split

## Import Env File

In [18]:
from dotenv import load_dotenv  

load_dotenv()
datasets_path = os.getenv('DATASET_PATH')
models_path = os.getenv('MODELS_PATH')

## Device Configuration

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


## Hyperparameters

In [5]:
num_classes = 2  
batch_size = 16
num_epochs = 20
learning_rate = 0.001
test_split_ratio = 0.2

## Data Preprocessing

In [6]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize sesuai input EfficientNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

## Data Loaders

In [ ]:
# Dataset
dataset = datasets.ImageFolder(root=datasets_path, transform=transform)

# Split dataset into training and testing
test_size = int(test_split_ratio * len(dataset))
train_size = len(dataset) - test_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# DataLoaders
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [12]:
# Cetak semua label kelas di dalam dataset
print("Daftar kelas dalam dataset:")
print(dataset.classes)

Daftar kelas dalam dataset:
['jogja', 'pekalongan']


## load Pre-Trained Model

In [10]:
# Load pre-trained EfficientNet
model = models.efficientnet_b0(pretrained=True)  # EfficientNet-B0 digunakan untuk transfer learning

# Customize the classifier (dense layer)
model.classifier = nn.Sequential(
    nn.Linear(model.classifier[1].in_features, 512),  # Dense layer baru
    nn.ReLU(),                                       # Activation function
    nn.Dropout(0.3),                                 # Dropout untuk regularisasi
    nn.Linear(512, num_classes)                      # Output layer untuk klasifikasi
)
model = model.to(device)

c:\Users\albia\anaconda3\envs\pytorch_env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\albia\anaconda3\envs\pytorch_env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to C:\Users\albia/.cache\torch\hub\checkpoints\efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:12<00:00, 1.72MB/s]


## Train The Model

In [11]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/20], Loss: 0.4821
Epoch [2/20], Loss: 0.2410
Epoch [3/20], Loss: 0.2026
Epoch [4/20], Loss: 0.1685
Epoch [5/20], Loss: 0.2838
Epoch [6/20], Loss: 0.2998
Epoch [7/20], Loss: 0.3377
Epoch [8/20], Loss: 0.0566
Epoch [9/20], Loss: 0.6984
Epoch [10/20], Loss: 0.1792
Epoch [11/20], Loss: 0.0694
Epoch [12/20], Loss: 0.0301
Epoch [13/20], Loss: 0.0492
Epoch [14/20], Loss: 0.0143
Epoch [15/20], Loss: 0.0094
Epoch [16/20], Loss: 0.2019
Epoch [17/20], Loss: 0.1435
Epoch [18/20], Loss: 0.2072
Epoch [19/20], Loss: 0.2477
Epoch [20/20], Loss: 0.1128


## Evaluation

In [13]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy: {100 * correct / total:.2f}%')

Accuracy: 87.50%


## Save The Model

In [ ]:
torch.save(model.state_dict(), models_path+'/CNN.pth')